In [65]:
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams["figure.figsize"] = (12, 16)

In [66]:
import pandas as pd

In [67]:
IN_PATH: str = '../results/manifolds/raw'
OUT_PATH: str = '../results/manifolds'

In [68]:
DATASETS: list = ['train', 'test']

In [69]:
MODELS: list = [ 'base', 'textattack', 'fabriceyhc', 'wakaka']

In [70]:
COLS_OF_INTEREST: list = ['mean', 'std', 'min', 'max']

In [71]:
### Load Datasets into memory

In [72]:
def convert_to_multi_col(df:pd.DataFrame, meta_col: str) -> pd.DataFrame:
    df.columns = pd.MultiIndex.from_product([[meta_col], df.columns], names=['model', 'metrics'])

    return df

In [73]:
analysis: dict = {
    data: pd.concat(
        [(
            pd
            .read_csv(f'{IN_PATH}/{data}.{model}.csv', index_col=0)
            .set_index(['dim'])
            .pipe(convert_to_multi_col, model)
        )
            for model in MODELS],
        axis=1)
    for data in DATASETS
}

In [74]:
for label, data in analysis.items():
    display(label, data)

'train'

model                  base                                        textattack  \
metrics dispersion_positive dispersion_negative  distance dispersion_positive   
dim                                                                             
768            1.643370e+06        1.591676e+06  1.259246        1.915401e+06   
576            1.644534e+06        1.592719e+06  1.194103        1.915778e+06   
384            1.644351e+06        1.592551e+06  1.199964        1.915490e+06   
192            1.643708e+06        1.591911e+06  1.219327        1.914635e+06   
96             1.642579e+06        1.590627e+06  1.246506        1.912659e+06   
48             1.640233e+06        1.588034e+06  1.293415        1.908444e+06   
24             1.634228e+06        1.581408e+06  1.443938        1.899243e+06   
12             1.622289e+06        1.568095e+06  1.648988        1.878269e+06   
6              1.599161e+06        1.537979e+06  1.852708        1.824655e+06   
3              1.561676e+06        1.476522e+06  1.520903        1.672867e+06   

model                                          fabriceyhc                      \
metrics dispersion_negative  distance dispersion_positive dispersion_negative   
dim                                                                             
768            1.867103e+06  7.820493        1.628460e+06        2.056547e+06   
576            1.867371e+06  7.822215        1.628916e+06        2.057076e+06   
384            1.867085e+06  7.824227        1.628829e+06        2.057008e+06   
192            1.866176e+06  7.829552        1.628542e+06        2.056760e+06   
96             1.864185e+06  7.843067        1.627981e+06        2.056267e+06   
48             1.859759e+06  7.871172        1.626742e+06        2.055172e+06   
24             1.850124e+06  7.931141        1.623751e+06        2.052485e+06   
12             1.828336e+06  8.063804        1.616275e+06        2.045783e+06   
6              1.774034e+06  8.383164        1.594092e+06        2.024674e+06   
3              1.627430e+06  9.144963        1.492467e+06        1.945725e+06   

model                           wakaka                                 
metrics   distance dispersion_positive dispersion_negative   distance  
dim                                                                    
768      22.864886        2.296890e+06        2.062966e+06  11.483087  
576      22.865054        2.297310e+06        2.063295e+06  11.485725  
384      22.865054        2.297080e+06        2.063062e+06  11.486758  
192      22.865390        2.296474e+06        2.062360e+06  11.488441  
96       22.865806        2.294904e+06        2.060776e+06  11.494536  
48       22.866762        2.291651e+06        2.057314e+06  11.506000  
24       22.869183        2.284118e+06        2.049246e+06  11.537372  
12       22.876083        2.266485e+06        2.030528e+06  11.608496  
6        22.898670        2.218734e+06        1.980252e+06  11.813304  
3        23.058022        2.088940e+06        1.840697e+06  12.382923

'test'

model                  base                                        textattack  \
metrics dispersion_positive dispersion_negative  distance dispersion_positive   
dim                                                                             
768            1.507873e+06        1.702410e+06  1.229414        1.786823e+06   
576            1.509076e+06        1.703589e+06  1.156741        1.787146e+06   
384            1.508934e+06        1.703403e+06  1.158986        1.786892e+06   
192            1.508509e+06        1.702874e+06  1.169901        1.786042e+06   
96             1.507134e+06        1.701512e+06  1.213615        1.784244e+06   
48             1.504606e+06        1.698812e+06  1.272759        1.780371e+06   
24             1.499124e+06        1.692158e+06  1.409630        1.771795e+06   
12             1.489605e+06        1.680789e+06  1.508193        1.752307e+06   
6              1.482086e+06        1.662280e+06  0.950279        1.702880e+06   
3              1.434469e+06        1.588580e+06  1.229867        1.560394e+06   

model                                          fabriceyhc                      \
metrics dispersion_negative  distance dispersion_positive dispersion_negative   
dim                                                                             
768            2.042893e+06  7.312816        1.833460e+06        2.608884e+06   
576            2.043185e+06  7.314414        1.833902e+06        2.609408e+06   
384            2.042884e+06  7.316451        1.833828e+06        2.609366e+06   
192            2.041993e+06  7.322656        1.833563e+06        2.609061e+06   
96             2.040015e+06  7.335378        1.832996e+06        2.608601e+06   
48             2.035766e+06  7.362444        1.831810e+06        2.607396e+06   
24             2.026320e+06  7.423075        1.828797e+06        2.604430e+06   
12             2.004841e+06  7.558836        1.821511e+06        2.597674e+06   
6              1.950692e+06  7.878849        1.799080e+06        2.574212e+06   
3              1.799460e+06  8.708559        1.706871e+06        2.483648e+06   

model                           wakaka                                 
metrics   distance dispersion_positive dispersion_negative   distance  
dim                                                                    
768      19.964325        2.091581e+06        2.231614e+06  11.398275  
576      19.964686        2.092026e+06        2.231988e+06  11.401153  
384      19.964686        2.091831e+06        2.231784e+06  11.401629  
192      19.965168        2.091119e+06        2.230950e+06  11.405117  
96       19.965624        2.089732e+06        2.229337e+06  11.409593  
48       19.966653        2.086472e+06        2.225610e+06  11.423860  
24       19.970807        2.079016e+06        2.217130e+06  11.455494  
12       19.978293        2.060904e+06        2.196837e+06  11.539209  
6        20.011916        2.013277e+06        2.144489e+06  11.756336  
3        20.243306        1.887557e+06        1.995884e+06  12.355372

In [75]:
### Calculate Centroid Distance and Cluster Dispersion

In [76]:
distances: dict = {
    label: (
        data
        .filter(regex=".*distance")
        .describe()
        .T
    )
    for label, data in analysis.items()
}

In [77]:
for label, data in distances.items():
    formatted: pd.DataFrame = (
        data
        [COLS_OF_INTEREST]
        .round(3)
    )
    formatted.to_csv(f'{OUT_PATH}/{label}.metric.distance.csv')
    formatted.style.to_latex(f'{OUT_PATH}/{label}.metric.distance.tex')
    display(label, formatted)
    # print(formatted.to_markdown())

'train'

,,mean,std,min,max
model,metrics,,,,
base,distance,1.388,0.224,1.194,1.853
textattack,distance,8.053,0.422,7.820,9.145
fabriceyhc,distance,22.889,0.060,22.865,23.058
wakaka,distance,11.629,0.284,11.483,12.383


'test'

,,mean,std,min,max
model,metrics,,,,
base,distance,1.230,0.151,0.950,1.508
textattack,distance,7.553,0.443,7.313,8.709
fabriceyhc,distance,20.000,0.087,19.964,20.243
wakaka,distance,11.555,0.303,11.398,12.355


In [78]:
dispersion: dict = {
    label: (
        data
        .filter(regex=".*dispersion")
        .describe()
        .T
    )
    for label, data in analysis.items()
}

In [79]:
for label, data in dispersion.items():
    formatted: pd.DataFrame = (
        data
        [['mean', 'std', 'min', 'max']]
        .round(3)
    )
    formatted.to_csv(f'{OUT_PATH}/{label}.metric.dispersion.csv')
    formatted.style.to_latex(f'{OUT_PATH}/{label}.metric.dispersion.tex')
    display(label, formatted)
    # print(formatted.to_markdown())

'train'

mean        std          min  \
model      metrics                                                    
base       dispersion_positive  1627613.104  27256.904  1561675.581   
           dispersion_negative  1571152.271  37430.363  1476522.285   
textattack dispersion_positive  1875744.028  76759.077  1672866.519   
           dispersion_negative  1827160.148  76015.529  1627429.920   
fabriceyhc dispersion_positive  1609605.553  42549.923  1492467.351   
           dispersion_negative  2040749.680  34859.701  1945724.828   
wakaka     dispersion_positive  2263258.640  65963.258  2088939.738   
           dispersion_negative  2027049.680  70413.880  1840696.849   

                                        max  
model      metrics                           
base       dispersion_positive  1644534.351  
           dispersion_negative  1592719.381  
textattack dispersion_positive  1915778.263  
           dispersion_negative  1867370.677  
fabriceyhc dispersion_positive  1628916.281  
           dispersion_negative  2057076.245  
wakaka     dispersion_positive  2297310.361  
           dispersion_negative  2063295.329

'test'

mean        std          min  \
model      metrics                                                    
base       dispersion_positive  1495141.591  23230.127  1434468.683   
           dispersion_negative  1683640.672  35953.293  1588580.038   
textattack dispersion_positive  1749889.361  71602.151  1560393.844   
           dispersion_negative  2002804.965  77080.489  1799460.053   
fabriceyhc dispersion_positive  1815581.831  39690.747  1706871.390   
           dispersion_negative  2591267.887  39343.675  2483647.913   
wakaka     dispersion_positive  2058351.422  64845.215  1887557.195   
           dispersion_negative  2193562.313  74640.670  1995884.354   

                                        max  
model      metrics                           
base       dispersion_positive  1509076.011  
           dispersion_negative  1703589.013  
textattack dispersion_positive  1787146.411  
           dispersion_negative  2043185.339  
fabriceyhc dispersion_positive  1833901.678  
           dispersion_negative  2609407.772  
wakaka     dispersion_positive  2092025.930  
           dispersion_negative  2231988.091